Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## AKS での負荷テスト

モデルを運用環境に展開したら、展開ターゲットが予想される負荷 (ユーザー数と予想される応答速度) を確実にサポートできるようにすることが重要です。これは、複数のユーザーに対するレコメンデーションを同時にサポートする必要がある運用システムの場合に特に重要です。同時ユーザー数が増加すると、レコメンデーション システムの負荷が大幅に増加する可能性があるため、不要なシステム障害やユーザーの応答時間の遅さを避けるために、運用システムの限界を理解する必要があります。

この種のロード テストを実行するために、ユーザー要求をさまざまな速度でシミュレートし、1 秒あたりの要求数やサービスの平均応答時間を確立するツールを利用できます。このノートブックでは、Azure Kubernetes Service (AKS) でデプロイされたモデルの負荷テストを実行するプロセスについて説明します。

このノートブックは、Azure Machine Learning ワークスペースからモデルをデプロイするために AKS Web サービスが使用されていることを前提としています。このアプローチの例は、[LightGBM 運用化ノートブック](lightgbm_criteo_o16n.ipynb)に用意されています。

[Locust](https://docs.locust.io/en/stable/)を使用して負荷テストを実行します。このツールの詳細についてはリンク先のドキュメントを参照してください。

In [1]:
import os
import subprocess
import sys
from tempfile import TemporaryDirectory
from urllib.parse import urlparse

sys.path.append('../..')

import requests

from azureml.core import Workspace
from azureml.core import VERSION as azureml_version
from azureml.core.webservice import AksWebservice

from reco_utils.dataset.criteo import get_spark_schema, load_pandas_df
from reco_utils.azureml.azureml_utils import get_or_create_workspace

# コア SDK バージョンを確認
print("Azure ML SDK version: {}".format(azureml_version))

Azure ML SDK version: 1.0.18


In [2]:
# 後で locust からのすべての出力をキャプチャするために、セルの幅を大きくします
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### 生成されたファイルの一時ディレクトリを作成する

In [3]:
TMP_DIR = TemporaryDirectory()

### AKS Service情報の取得

In [4]:
# 展開済みサービス名と一致している必要があります
SERVICE_NAME = 'lightgbm-criteo'

In [5]:
ws = get_or_create_workspace()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Found the config file in: C:\Users\scgraham\repos\Recommenders\notebooks\05_operationalize\aml_config\config.json
Wrote the config file config.json to: C:\Users\scgraham\repos\Recommenders\notebooks\05_operationalize\aml_config\config.json


In [6]:
aks_service = AksWebservice(ws, name=SERVICE_NAME)

In [7]:
# スコアリング URIの取得
url = aks_service.scoring_uri
parsed_url = urlparse(url)

# aks_service のキーの 1 つを使用して認証をセットアップ
headers = dict(Authorization='Bearer {}'.format(aks_service.get_keys()[0]))

### テスト用のサンプル データを取得する

In [8]:
# いくつのサンプル データを取得
df = load_pandas_df(size='sample')

8.79MB [00:04, 1.93MB/s]                                                                                                                                                                                                                                                   


In [9]:
data = df.iloc[0, :].to_json()
print(data)

{"label":0,"int00":1.0,"int01":1,"int02":5.0,"int03":0.0,"int04":1382.0,"int05":4.0,"int06":15.0,"int07":2.0,"int08":181.0,"int09":1.0,"int10":2.0,"int11":null,"int12":2.0,"cat00":"68fd1e64","cat01":"80e26c9b","cat02":"fb936136","cat03":"7b4723c4","cat04":"25c83c98","cat05":"7e0ccccf","cat06":"de7995b8","cat07":"1f89b562","cat08":"a73ee510","cat09":"a8cd5504","cat10":"b2cb9c98","cat11":"37c9c164","cat12":"2824a5f6","cat13":"1adce6ef","cat14":"8ba8b39a","cat15":"891b62e7","cat16":"e5ba7672","cat17":"f54016b9","cat18":"21ddcdc9","cat19":"b1252a9d","cat20":"07b5194c","cat21":null,"cat22":"3a171ecb","cat23":"c5c50484","cat24":"e8b83407","cat25":"9727dd16"}


In [10]:
# aks サービスが実行されていること、期待される結果が提供されることを確認
aks_service.run(data)

'{"result": 0.35952275816753043}'

In [11]:
# サービスへの HTTP 要求が機能することを確認
response = requests.post(url=url, json=data, headers=headers)
print(response.json())

{"result": 0.35952275816753043}


### LocustFile のセットアップ

Locust は、ユーザーの動作を制御するローカスト ファイル (デフォルトで locustfile.py) を使用します。

この例では、ユーザーが起動するたびに実行するタスクをカプセル化する UserBehavior クラスを作成します。ここではサービスがサンプルデータで要求を処理できることを保証することにのみ興味があるので、使用される唯一のタスクは、上記の手動で行われたような単純なポストリクエストであるスコアリングタスクです。

次のクラスは、ユーザーのインスタンス化方法を定義します。この場合、ホストサーバーとの http セッションを開始して定義されたタスクを実行するユーザーを作成します。タスクは、少しの時間待ちの後に繰り返されます。この待機時間は、最小待機時間と最大待機時間 (ミリ秒) の間に均一なランダム サンプルを作成することによって決定されます。

In [12]:
locustfile = """
from locust import HttpLocust, TaskSet, task


class UserBehavior(TaskSet):
    @task
    def score(self):
        self.client.post("{score_url}", json='{data}', headers={headers})


class WebsiteUser(HttpLocust):
    task_set = UserBehavior
    # min and max time to wait before repeating task
    min_wait = 1000
    max_wait = 2000
""".format(data=data, headers=headers, score_url=parsed_url.path)

locustfile_path = os.path.join(TMP_DIR.name, 'locustfile.py')
with open(locustfile_path, 'w') as f:
    f.write(locustfile)

次の手順では、locust 負荷テスト ツールを開始します。Web インターフェイスで実行することも、コマンドラインから直接実行することもできます。今回の場合はコマンドラインから実行し、同時ユーザー数、ユーザーの生成速度、テストの実行期間を指定します。これらのオプションはすべて、Web インターフェイス GUI を介して制御できるだけでなく、障害に関するより多くの情報を提供することもできるので、より高度な使用方法についてはドキュメントを読むことをお勧めします。ここでは、テストを実行し、要約結果をキャプチャします。

In [13]:
cmd = "locust -H {host} -f {path} --no-web -c {users} -r {rate} -t {duration} --only-summary".format(
    host='{url.scheme}://{url.netloc}'.format(url=parsed_url),
    path=locustfile_path,
    users=200,  # 同時接続ユーザー数
    rate=10,  # ハッチ率 (ユーザー / 秒)
    duration='1m',  # テスト期間
)
process = subprocess.run(cmd, shell=True, stderr=subprocess.PIPE)
print(process.stderr.decode('utf-8'))

[2019-05-28 12:36:31,630] 9821192-1116/INFO/locust.main: Run time limit set to 60 seconds
[2019-05-28 12:36:31,631] 9821192-1116/INFO/locust.main: Starting Locust 0.11.0
[2019-05-28 12:36:31,631] 9821192-1116/INFO/locust.runners: Hatching and swarming 200 clients at the rate 10 clients/s...
[2019-05-28 12:36:51,864] 9821192-1116/INFO/locust.runners: All locusts hatched: WebsiteUser: 200
[2019-05-28 12:37:30,701] 9821192-1116/INFO/locust.main: Time limit reached. Stopping Locust.
[2019-05-28 12:37:30,707] 9821192-1116/INFO/locust.main: Shutting down (exit code 0), bye.
[2019-05-28 12:37:30,707] 9821192-1116/INFO/locust.main: Cleaning up runner...
[2019-05-28 12:37:30,738] 9821192-1116/INFO/locust.main: Running teardowns...
 Name                                                          # reqs      # fails     Avg     Min     Max  |  Median   req/s
--------------------------------------------------------------------------------------------------------------------------------------------
 

### 負荷テストの結果

上記では、応答時間に関する要求、失敗、統計の数、およびサーバーが処理している 1 秒あたりの要求数を確認できます。

2 行目は、負荷が応答速度にどのような影響を与えているか、およびパフォーマンスに影響を与える外れ値があるかどうかを把握するのに役立つ応答時間の分布を示しています。

### 一時ディレクトリのクリーンアップ

In [14]:
TMP_DIR.cleanup()